In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import warnings
warnings.filterwarnings('ignore')
import os

## [0] Environment Setup

setup for running 
- created virtual environment in conda 
> conda create -n CCB_BioPlexPy python=3.9.12
- installed **pandas**
> conda install -c anaconda pandas
- installed **requests**
> conda install -c anaconda requests
- installed **anndata**
> conda install anndata -c bioconda
- installed **jupyter notebook**
> conda install -c conda-forge notebook
- installed **networkx**
> pip install networkx[default]
- installed **biopython**
> conda install -c conda-forge biopython
- installed **pypdb**
> pip install pypdb

## [1] *Function* to retrieve interactions data

Column Descriptions

GeneA: Entrez Gene ID for the first interacting protein

GeneB: Entrez Gene ID for the second interacting protein

UniprotA: Uniprot ID for the first interacting protein

UniprotB: Uniprot ID for the second interacting protein

SymbolA: Symbol for the first interacting protein

SymbolB: Symbol for the second interacting protein

p(Wrong ID): Probability of wrong protein ID (CompPASS-Plus)

p(NotInteractor): Probability of nonspecific background (CompPASS-Plus)

p(Interactor): Probability of high-confidence interaction (CompPASS-Plus)

In [11]:
import pandas as pd

def getBioPlex(cell_line, version):
    '''
    Load BioPlex interactions data.
    
    This function loads BioPlex PPI data for
    cell lines HEK293T and HCT116, note we
    only have version 1.0 for HCT116 cells.
    
    Parameters
    ----------
    cell_line : str
        Takes input ['293T','HCT116'].
    version : str
        Takes input ['3.0','1.0','2.0'].
    
    Returns
    -------
    Pandas DataFrame
        A dataframe with each row corresponding to a PPI interaction.
    
    Examples
    --------
    >>> bp_293t = getBioPlex('293T', '1.0')
    >>> bp_hct116 = getBioPlex('HCT116', '1.0')
    '''
    if (f'{cell_line}.{version}' not in 
        ['293T.1.0','293T.2.0','293T.3.0','HCT116.1.0']):
        print('dataset not available for this Cell Line - Version')
        
    else:
        if f'{cell_line}.{version}' == '293T.1.0':
            file_ext = 'interactionList_v2'
        elif f'{cell_line}.{version}' == '293T.2.0':
            file_ext = 'interactionList_v4a'
        elif f'{cell_line}.{version}' == '293T.3.0':
            file_ext = '293T_Network_10K_Dec_2019'
        elif f'{cell_line}.{version}' == 'HCT116.1.0':
            file_ext = 'HCT116_Network_5.5K_Dec_2019'

        BioPlex_interactions_df = pd.read_csv(
                f"https://bioplex.hms.harvard.edu/data/BioPlex_{file_ext}.tsv", 
                sep = '\t')
        
    # if pulling 293T cell line version 1.0 or 2.0, change column names to 
    # standardize across datasets for input into other functions
    if (cell_line == '293T') and (version == '1.0'):
        BioPlex_interactions_df.rename({'Gene A':'GeneA','Gene B':'GeneB',
            'Uniprot A':'UniprotA','Uniprot B':'UniprotB',
            'Symbol A':'SymbolA','Symbol B':'SymbolB','p(Wrong)':'pW',
            'p(No Interaction)':'pNI','p(Interaction)':'pInt'}, 
            axis = 1, inplace = True)

    if (cell_line == '293T') and (version == '2.0'):
        BioPlex_interactions_df.rename({'p(Wrong)':'pW',
            'p(No Interaction)':'pNI','p(Interaction)':'pInt'}, 
            axis = 1, inplace = True)
    
    return BioPlex_interactions_df

Below is an attempt to modify the function above to add HEK293T + HCT116 cell line edges and return all edges found in both cell lines

- concatenate dataframes and take union (delete redundant rows)
- some protein-A/protein-B & protein-B/protein-A switched, account for this
- additional column (HEK only, HCT only, HEK & HCT)

Turns out this is harder than expected so it will have to wait for version 2 of this package

In [89]:
import pandas as pd
from collections import Counter

# load 293T cell line bp interactions
bp_293T_v3_df = pd.read_csv(
        'https://bioplex.hms.harvard.edu/data/'
        'BioPlex_293T_Network_10K_Dec_2019.tsv', 
        sep = '\t')

# load HCT116 cell line bp interactions
bp_HCT116_v1_df = pd.read_csv(
        'https://bioplex.hms.harvard.edu/data/'
        'BioPlex_HCT116_Network_5.5K_Dec_2019.tsv', 
        sep = '\t')

# extract information for the 293T cell line
gene_pairs_293T_v3 = []
gene_pairs_reversed_293T_v3 = []
uniprot_map_293T_v3 = {}
symbol_map_293T_v3 = {}
for gene_A, gene_B, uniprot_A, uniprot_B, symbol_A, symbol_B in zip(bp_293T_v3_df.GeneA,bp_293T_v3_df.GeneB,bp_293T_v3_df.UniprotA,bp_293T_v3_df.UniprotB,bp_293T_v3_df.SymbolA,bp_293T_v3_df.SymbolB):
    gene_pairs_293T_v3.append(f'{gene_A}_{gene_B}')
    gene_pairs_reversed_293T_v3.append(f'{gene_B}_{gene_A}')
    uniprot_map_293T_v3[f'{gene_A}_{gene_B}'] = f'{uniprot_A}_{uniprot_B}'
    symbol_map_293T_v3[f'{gene_A}_{gene_B}'] = f'{symbol_A}_{symbol_B}'

# extract information for the HCT116 cell line
gene_pairs_HCT116_v1 = []
gene_pairs_reversed_HCT116_v1 = []
uniprot_map_HCT116_v1 = {}
symbol_map_HCT116_v1 = {}
for gene_A, gene_B, uniprot_A, uniprot_B, symbol_A, symbol_B in zip(bp_HCT116_v1_df.GeneA,bp_HCT116_v1_df.GeneB,bp_HCT116_v1_df.UniprotA,bp_HCT116_v1_df.UniprotB,bp_HCT116_v1_df.SymbolA,bp_HCT116_v1_df.SymbolB):
    gene_pairs_HCT116_v1.append(f'{gene_A}_{gene_B}')
    gene_pairs_reversed_HCT116_v1.append(f'{gene_B}_{gene_A}')
    uniprot_map_HCT116_v1[f'{gene_A}_{gene_B}'] = f'{uniprot_A}_{uniprot_B}'
    symbol_map_HCT116_v1[f'{gene_A}_{gene_B}'] = f'{symbol_A}_{symbol_B}'
    
gene_pairs_293T_and_HCT116 = (gene_pairs_293T_v3 + 
                                gene_pairs_HCT116_v1)

# get rid of redundant pairs to parse through
gene_pairs_293T_and_HCT116 = list(set(gene_pairs_293T_and_HCT116))

# convert lists to sets for faster checking
gene_pairs_293T_v3 = set(gene_pairs_293T_v3)
gene_pairs_reversed_293T_v3 = set(gene_pairs_reversed_293T_v3)
gene_pairs_HCT116_v1 = set(gene_pairs_HCT116_v1)
gene_pairs_reversed_HCT116_v1 = set(gene_pairs_reversed_HCT116_v1)

# iterate through each unique gene pair found in either cell line 
# and extract relevant information
Gene_A = []
Gene_B = []
Uniprot_A = []
Uniprot_B = []
Symbol_A = []
Symbol_B = []
cell_line_col = []
for gene_pair in gene_pairs_293T_and_HCT116:
    
    Gene_A.append(gene_pair.split('_')[0])
    Gene_B.append(gene_pair.split('_')[1])
    
    # check if interaction is found in both cell lines
    if (gene_pair in gene_pairs_293T_v3.union(gene_pairs_reversed_293T_v3)) and (gene_pair in gene_pairs_HCT116_v1.union(gene_pairs_reversed_HCT116_v1)):
    
        cell_line_col.append('293T_and_HCT116')
        
        # take mappings from 293T cell line or HCT116 cell line
        if gene_pair in gene_pairs_293T_v3:
            Uniprot_A.append(uniprot_map_293T_v3[gene_pair].split('_')[0])
            Uniprot_B.append(uniprot_map_293T_v3[gene_pair].split('_')[1])
            Symbol_A.append(symbol_map_293T_v3[gene_pair].split('_')[0])
            Symbol_B.append(symbol_map_293T_v3[gene_pair].split('_')[1])
            
        elif gene_pair in gene_pairs_HCT116_v1:
            Uniprot_A.append(uniprot_map_HCT116_v1[gene_pair].split('_')[0])
            Uniprot_B.append(uniprot_map_HCT116_v1[gene_pair].split('_')[1])
            Symbol_A.append(symbol_map_HCT116_v1[gene_pair].split('_')[0])
            Symbol_B.append(symbol_map_HCT116_v1[gene_pair].split('_')[1])
            
        else:
            print('oh no...something is wrong')

Examples below of function use

In [12]:
bp_293t = getBioPlex('293T', '2.0')

In [13]:
bp_293t

,GeneA,GeneB,UniprotA,UniprotB,SymbolA,SymbolB,pW,pNI,pInt
0,100,728378,P00813,A5A3E0,ADA,POTEF,2.380858e-09,3.318559e-04,0.999668
1,100,345651,P00813,Q562R1,ADA,ACTBL2,9.786437e-18,2.119144e-01,0.788086
2,222389,708,Q8N7W2,Q07021,BEND7,C1QBP,2.962215e-17,5.644512e-03,0.994355
3,222389,4038,Q8N7W2,O75096,BEND7,LRP4,3.302994e-10,2.802596e-04,0.999720
4,645121,3312,Q6ZMN8,P11142,CCNI2,HSPA8,2.060285e-16,3.623477e-02,0.963765
...,...,...,...,...,...,...,...,...,...
56548,652968,729438,Q8WTX7,A6NHX0,GATSL3,GATSL2,5.801810e-09,2.783396e-09,1.000000
56549,652968,54468,Q8WTX7,Q9NXC5,GATSL3,MIOS,1.016003e-08,2.169769e-05,0.999978
56550,652968,79726,Q8WTX7,Q6PJI9,GATSL3,WDR59,6.225445e-16,1.575947e-02,0.984241
56551,652968,84219,Q8WTX7,Q96S15-2,GATSL3,WDR24,1.084232e-10,8.419503e-06,0.999992


In [14]:
bp_hct116 = getBioPlex('HCT116', '1.0')

In [15]:
bp_hct116

,GeneA,GeneB,UniprotA,UniprotB,SymbolA,SymbolB,pW,pNI,pInt
0,88455,50649,Q8IZ07,Q9NR80-4,ANKRD13A,ARHGEF4,3.959215e-04,0.000033,0.999571
1,88455,115106,Q8IZ07,Q96CS2,ANKRD13A,HAUS1,4.488473e-02,0.001935,0.953181
2,88455,23086,Q8IZ07,Q8NEV8-2,ANKRD13A,EXPH5,7.402394e-05,0.000930,0.998996
3,88455,54930,Q8IZ07,Q9H6D7,ANKRD13A,HAUS4,9.180959e-07,0.000128,0.999871
4,88455,79441,Q8IZ07,Q68CZ6,ANKRD13A,HAUS3,8.709394e-07,0.001495,0.998504
...,...,...,...,...,...,...,...,...,...
70961,55118,100509631,Q9NQ79-2,F5H8C6,CRTAC1,LOC100509631,1.619493e-01,0.003972,0.834079
70962,1813,2177,P14416-3,Q9BXW9-2,DRD2,FANCD2,1.538024e-04,0.174397,0.825449
70963,1813,22870,P14416-3,Q9UPN7,DRD2,PPP6R1,1.464248e-08,0.072449,0.927551
70964,1813,23243,P14416-3,O15084-1,DRD2,ANKRD28,2.105728e-04,0.202417,0.797373


## [2] *Function* to retrieve HEK293 RNAseq expression data

In [16]:
import io
import requests
import anndata as ad
import pandas as pd

def getGSE122425():
    '''
    Retrieve HEK293 RNAseq expression data.
    
    Returns
    -------
    adata : AnnData object
        SummarizedExperiment of HEK293 raw count with an 
        added layer storing rpkm.
    
    Examples
    --------
    >>> HEK293_adata = getGSE122425()
    '''
    # specify URL where data is stored
    baseURL = ('https://ftp.ncbi.nlm.nih.gov/geo/series/GSE122nnn/'
               'GSE122425/suppl/')
    filename = 'GSE122425_all.counts.293_vs_293NK.edgeR_all.xls.gz'
    outFilePath = filename[:-3]

    # stream the file as bytes into memory using 
    # io.bytesIO and decompress using pandas
    response = requests.get(baseURL + filename)
    content = response.content
    GSE122425_df = pd.read_csv(io.BytesIO(content), 
                               sep='\t', compression='gzip')

    # create annot for observations (rows)
    obs = GSE122425_df.loc[:,['gene_id','GeneSymbol','KO','GO','length']]
    obs.set_index('gene_id',inplace=True)
    obs.rename(mapper={'GeneSymbol':'SYMBOL'}, inplace=True, axis=1)

    # we have raw counts and rpkms here in one matrix
    # raw counts
    raw_X = (GSE122425_df.loc[:,
                              ['HEK293NK-SEQ1','HEK293NK-SEQ2',
                               'HEK293NK-SEQ3','HEK293-SEQ1','HEK293-SEQ2',
                               'HEK293-SEQ3']].values)
    # annot for variables (cols)
    raw_var = pd.DataFrame(index=['NK.1','NK.2','NK.3','WT.1','WT.2','WT.3'])
    
    # convert to AnnData object (default datatype is 'float32')
    adata = ad.AnnData(raw_X, obs=obs, var=raw_var, dtype='int32')

    # store rpkms as a layer
    rpkm_X = (GSE122425_df.loc[:,['HEK293NK-SEQ1_RPKM','HEK293NK-SEQ2_RPKM',
                                  'HEK293NK-SEQ3_RPKM','HEK293-SEQ1_RPKM',
                                  'HEK293-SEQ2_RPKM',
                                  'HEK293-SEQ3_RPKM']].values)
    adata.layers["rpkm"] = rpkm_X

    return adata

In [17]:
HEK293_adata = getGSE122425()

In [18]:
HEK293_adata

AnnData object with n_obs × n_vars = 57905 × 6 
    obs: 'SYMBOL', 'KO', 'GO', 'length'
    layers: 'rpkm'

rows (observations)

In [19]:
print(HEK293_adata.obs_names[:10].tolist())

['ENSG00000223972', 'ENSG00000227232', 'ENSG00000243485', 'ENSG00000237613', 'ENSG00000268020', 'ENSG00000240361', 'ENSG00000186092', 'ENSG00000238009', 'ENSG00000239945', 'ENSG00000233750']


columns (variables)

In [20]:
print(HEK293_adata.var_names.tolist())

['NK.1', 'NK.2', 'NK.3', 'WT.1', 'WT.2', 'WT.3']


matrix w/ counts

In [21]:
print(HEK293_adata.X)

[[   0    0    2    1    2    2]
 [ 705  812 1121  732  690  804]
 [   0    0    0    0    0    2]
 ...
 [   0    0    0    0    0    0]
 [   0    0    0    0    0    0]
 [   0    0    0    0    0    0]]


matrix w/ rpkm

In [22]:
print(HEK293_adata.layers["rpkm"])

[[0.   0.   0.01 0.01 0.01 0.01]
 [4.77 5.21 6.8  5.43 5.07 5.39]
 [0.   0.   0.   0.   0.   0.04]
 ...
 [0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.  ]]


## [3] *Function* to retrieve CORUM complex data

In [23]:
import pandas as pd
import io
import requests

def getCorum(complex_set = 'all', organism = 'Human'):
    '''
    Functionality for retrieving the CORUM protein complex data.

    Parameters
    ----------
    complex_set : str
        Takes input ['all','core','splice'] (default 'all').
    organism : str
        Takes input ['Bovine','Dog','Hamster','Human','MINK','Mammalia',
        'Mouse','Pig','Rabbit','Rat'] (default 'Human').

    Returns
    -------
    Pandas DataFrame
        A dataframe with each row corresponding to a CORUM complex.

    Examples
    --------
    >>> CORUM_df = getCorum()
    >>> CORUM_df = getCorum('core', 'Human')
    '''
    # specify URL where data is stored
    baseURL = 'https://mips.helmholtz-muenchen.de/corum/download/'
    filename = f'{complex_set}Complexes.txt.zip'
    outFilePath = filename[:-4]

    # stream the file as bytes into memory using
    # io.bytesIO and decompress using pandas
    response = requests.get(baseURL + filename)
    content = response.content
    CORUM_df = pd.read_csv(io.BytesIO(content), sep='\t', compression='zip')

    # filter to keep only CORUM sets for a specific organism
    CORUM_df = CORUM_df[CORUM_df.Organism == organism]
    CORUM_df.reset_index(inplace = True, drop = True)
    
    return CORUM_df

In [24]:
Corum_DF = getCorum('core', 'Human')

In [25]:
Corum_DF.head()

,ComplexID,ComplexName,Organism,Synonyms,Cell line,subunits(UniProt IDs),subunits(Entrez IDs),Protein complex purification method,GO ID,GO description,FunCat ID,FunCat description,subunits(Gene name syn),Complex comment,Disease comment,SWISSPROT organism,Subunits comment,subunits(Gene name),PubMed ID,subunits(Protein name)
0,1,BCL6-HDAC4 complex,Human,None,None,P41182;P56524,604;9759,MI:0007-anti tag coimmunoprecipitation,GO:0006265;GO:0045892;GO:0051276;GO:0030183;GO...,DNA topological change;negative regulation of ...,10.01.09.05;11.02.03.04.03;42.10.03;43.03.07.0...,DNA conformation modification (e.g. chromatin)...,BCL5 LAZ3 ZBTB27 ZNF51;KIAA0288,Transcriptional repression by BCL6 is thought ...,None,Homo sapiens (Human);Homo sapiens (Human),None,BCL6;HDAC4,11929873,B-cell lymphoma 6 protein;Histone deacetylase 4
1,2,BCL6-HDAC5 complex,Human,None,None,P41182;Q9UQL6,604;10014,MI:0007-anti tag coimmunoprecipitation,GO:0006265;GO:0045892;GO:0051276;GO:0030183;GO...,DNA topological change;negative regulation of ...,10.01.09.05;11.02.03.04.03;42.10.03;43.03.07.0...,DNA conformation modification (e.g. chromatin)...,BCL5 LAZ3 ZBTB27 ZNF51;KIAA0600,Transcriptional repression by BCL6 is thought ...,None,Homo sapiens (Human);Homo sapiens (Human),None,BCL6;HDAC5,11929873,B-cell lymphoma 6 protein;Histone deacetylase 5
2,3,BCL6-HDAC7 complex,Human,None,None,P41182;Q8WUI4,604;51564,MI:0007-anti tag coimmunoprecipitation,GO:0006265;GO:0045892;GO:0051276;GO:0030183;GO...,DNA topological change;negative regulation of ...,10.01.09.05;11.02.03.04.03;42.10.03;43.03.07.0...,DNA conformation modification (e.g. chromatin)...,BCL5 LAZ3 ZBTB27 ZNF51;HDAC7A,Transcriptional repression by BCL6 is thought ...,None,Homo sapiens (Human);Homo sapiens (Human),None,BCL6;HDAC7,11929873,B-cell lymphoma 6 protein;Histone deacetylase 7
3,4,Multisubunit ACTR coactivator complex,Human,None,None,Q09472;Q92793;Q92831;Q9Y6Q9,2033;1387;8850;8202,MI:0004-affinity chromatography technologies;M...,GO:0045893;GO:0023052;GO:0005634,"positive regulation of transcription, DNA-temp...",11.02.03.04.01;30.01;70.10,transcription activation;cellular signalling;n...,"P300;CBP;PCAF;AIB1, BHLHE42, RAC3, TRAM1, ACTR",Cofactor ACTR binds directly nuclear receptors...,None,Homo sapiens (Human);Homo sapiens (Human);Homo...,None,EP300;CREBBP;KAT2B;NCOA3,9267036,Histone acetyltransferase p300;CREB-binding pr...
4,11,BLOC-3 (biogenesis of lysosome-related organel...,Human,None,None,Q92902;Q9NQG7,3257;89781,MI:0019- coimmunoprecipitation; MI:0029- cosed...,GO:0007032;GO:0007040;GO:0007033,endosome organization;lysosome organization;va...,42.22;42.25,endosome;vacuole or lysosome,HPS;KIAA1667,The results suggest that HPS1 and HPS4 are com...,HPS1-7 are involved in Hermansky-Pudlak syndro...,Homo sapiens (Human);Homo sapiens (Human),None,HPS1;HPS4,12847290,Hermansky-Pudlak syndrome 1 protein;Hermansky-...


In [26]:
Corum_DF.shape

(2417, 20)